# Install necessary packages

In [2]:
!pip install --upgrade pip

Defaulting to user installation because normal site-packages is not writeable
  Using cached pip-24.3.1-py3-none-any.whl.metadata (3.7 kB)
Using cached pip-24.3.1-py3-none-any.whl (1.8 MB)



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: To modify pip, please run the following command:
C:\Program Files\Python312\python.exe -m pip install --upgrade pip


In [3]:
!pip install -U torch numpy==1.26 pandas scikit-learn plotly nltk transformers==4.46.3 sentence-transformers einops datasets gradio networkx umap-learn ipywidgets

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


# Import Libraries

In [4]:
import os
import pandas as pd
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModel
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.neighbors import NearestNeighbors
import plotly.express as px
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.tokenize import PunktSentenceTokenizer
from sklearn.manifold import TSNE
from IPython.display import display
import umap
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
import nltk
from nltk.tokenize import sent_tokenize
from sklearn.preprocessing import normalize
import re

from IPython.display import display, HTML
# Tomas Mazak's workaround
import plotly
plotly.offline.init_notebook_mode()
display(HTML(
"""
<script type="text/javascript" async
src="https://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.1/MathJax.js?config=TeX-M">
"""
))
##

# Download NLTK resources

In [5]:
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\antoi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\antoi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\antoi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
# Define stopwords
stop_words = set(stopwords.words('french'))

def remove_stopwords(text):
    # Tokenize into words
    words = word_tokenize(text.lower())
    # Remove stopwords and non-alphabetic tokens
    filtered_words = [word for word in words if word.isalpha() and word not in stop_words]
    return ' '.join(filtered_words)

In [7]:
def tokenize_sentences(text):
    sentences = sent_tokenize(text, language='french')
    return ' '.join(sentences)

In [8]:
def preprocess_text(text):
    # Remove stopwords
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    remove_stopwords(text)
    return text.strip()
    return no_stopwords

# Load the CSV files to inspect their contents

In [9]:
current_dir = os.path.join(os.getcwd(), 'data')

In [10]:
program_df = pd.read_csv(os.path.join(current_dir, 'Program.csv'))
program_df['title'] = program_df['title'].str.replace(r'<[^>]*>', '', regex=True)
program_course_df = pd.read_csv(os.path.join(current_dir, 'ProgramCourse.csv'))
program_type_df = pd.read_csv(os.path.join(current_dir, 'ProgramType.csv'))
course_df = pd.read_csv(os.path.join(current_dir, 'Course.csv'))

# Load MiniLM model and tokenizer for generating embeddings

In [11]:
# tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")
# model = AutoModel.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
model = AutoModel.from_pretrained("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
# tokenizer = AutoTokenizer.from_pretrained("nvidia/NV-Embed-v2", trust_remote_code=True)
# model = AutoModel.from_pretrained("nvidia/NV-Embed-v2", trust_remote_code=True)
# # Move model to GPU if available for faster computation
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(250037, 384, padding_idx=0)
    (position_embeddings): Embedding(512, 384)
    (token_type_embeddings): Embedding(2, 384)
    (LayerNorm): LayerNorm((384,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=384, out_features=384, bias=True)
            (key): Linear(in_features=384, out_features=384, bias=True)
            (value): Linear(in_features=384, out_features=384, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=384, out_features=384, bias=True)
            (LayerNorm): LayerNorm((384,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=Fals

# Define a Function to Generate Embeddings

In [12]:
# Initialize device outside the embedding function
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(250037, 384, padding_idx=0)
    (position_embeddings): Embedding(512, 384)
    (token_type_embeddings): Embedding(2, 384)
    (LayerNorm): LayerNorm((384,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=384, out_features=384, bias=True)
            (key): Linear(in_features=384, out_features=384, bias=True)
            (value): Linear(in_features=384, out_features=384, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=384, out_features=384, bias=True)
            (LayerNorm): LayerNorm((384,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=Fals

In [13]:
def generate_embeddings(
    text_list, tokenizer, model, device=device, batch_size=32, pooling='mean', preprocess_fn=None
):
    """
    Generate embeddings for a list of texts using the specified pooling strategy.

    Args:
        text_list (list of str): List of input texts.
        tokenizer (transformers.AutoTokenizer): Tokenizer corresponding to the model.
        model (transformers.AutoModel): Pre-trained language model.
        device (torch.device): Device to perform computations on.
        batch_size (int, optional): Number of texts to process per batch. Defaults to 32.
        pooling (str, optional): Pooling strategy ('mean', 'max', 'concat'). Defaults to 'mean'.
        preprocess_fn (callable, optional): Preprocessing function for text. Defaults to None.

    Returns:
        np.ndarray: Generated embeddings.
    """
    if pooling not in ['mean', 'max', 'concat']:
        raise ValueError(f"Unsupported pooling type '{pooling}'. Choose from 'mean', 'max', 'concat'.")
    
    model.eval()  # Set model to evaluation mode
    embeddings = []

    # Preprocess text if a function is provided
    if preprocess_fn:
        text_list = [preprocess_fn(text) for text in text_list]

    with torch.no_grad():  # Disable gradient calculation
        for i in range(0, len(text_list), batch_size):
            batch_texts = text_list[i:i + batch_size]

            # Tokenize each text into sentences if too long
            tokenized_texts = [
                sent_tokenize(text) if len(text.split()) > 512 else [text] for text in batch_texts
            ]
            
            # Embed each sentence separately and aggregate
            batch_embeddings = []
            for sentences in tokenized_texts:
                # Tokenize and process sentences
                tokens = tokenizer(sentences, return_tensors='pt', padding=True, truncation=True, max_length=512)
                tokens = {k: v.to(device) for k, v in tokens.items()}
                outputs = model(**tokens)

                # Pool embeddings at sentence level
                if pooling == 'mean':
                    sentence_embeddings = outputs.last_hidden_state.mean(dim=1)
                elif pooling == 'max':
                    sentence_embeddings = outputs.last_hidden_state.max(dim=1).values
                elif pooling == 'concat':
                    mean_pool = outputs.last_hidden_state.mean(dim=1)
                    max_pool = outputs.last_hidden_state.max(dim=1).values
                    sentence_embeddings = torch.cat((mean_pool, max_pool), dim=1)
                
                # Aggregate sentence embeddings
                text_embedding = sentence_embeddings.mean(dim=0)
                batch_embeddings.append(text_embedding)

            # Append batch embeddings
            embeddings.append(torch.stack(batch_embeddings).cpu())

    # Concatenate all batch embeddings
    embeddings = torch.cat(embeddings, dim=0)

    # Normalize embeddings
    embeddings = normalize(embeddings.numpy(), axis=1)

    print(f"Generated embeddings with '{pooling}' pooling. Embedding dimension: {embeddings.shape[1]}")

    return embeddings

# Generate Embeddings for Programs

In [14]:
# List of columns to include
columns_to_include = [
    'title', 'code', 'cycle', 'url', 'id'
]

# Ensure all columns are strings and handle NaN values
for col in columns_to_include:
    program_df[col] = program_df[col].astype(str).fillna('')

# Concatenate the columns into a single string for each program
program_texts = program_df[columns_to_include].apply(lambda x: ' '.join(x), axis=1)

# Generate embeddings for the combined texts
program_embeddings = generate_embeddings(program_texts.tolist(), tokenizer, model)

# Add embeddings to program_df
program_df['vector'] = list(program_embeddings)


Generated embeddings with 'mean' pooling. Embedding dimension: 384


# Embeddings using PCA

In [15]:
pca = pca = PCA(n_components=2, random_state=42)
program_pca_result = pca.fit_transform(program_embeddings)
program_df['pca-one-program'] = program_pca_result[:, 0]
program_df['pca-two-program'] = program_pca_result[:, 1]

# Embeddings using t-SNE

In [16]:
tsne = TSNE(n_components=2, perplexity=30, max_iter=1000, random_state=42)
program_tsne_result = tsne.fit_transform(program_embeddings)
program_df['tsne-one-program'] = program_tsne_result[:, 0]
program_df['tsne-two-program'] = program_tsne_result[:, 1]

# UMAP

In [17]:
# Initialize UMAP with desired parameters
umap_projection = umap.UMAP(n_components=2, n_neighbors=15, min_dist=0.1)

# Fit and transform the program embeddings
program_umap_result = umap_projection.fit_transform(program_embeddings)

# Add UMAP results to your DataFrame
program_df['umap-one-program'] = program_umap_result[:, 0]
program_df['umap-two-program'] = program_umap_result[:, 1]

#  Generate Embeddings for Courses

In [18]:
# List of columns to include
columns_to_include = [
    'code', 'title', 'description', 'cycle', 'credits'
]

# Ensure all columns are strings and handle NaN values
for col in columns_to_include:
    course_df[col] = course_df[col].astype(str).fillna('')

# Concatenate the columns into a single string for each program
course_texts = course_df[columns_to_include].apply(lambda x: ' '.join(x), axis=1)

# Generate embeddings for the combined texts
course_embeddings = generate_embeddings(course_texts.tolist(), tokenizer, model)

# Add embeddings to program_df
course_df['vector'] = list(course_embeddings)

vectors = np.stack(course_df['vector'].values)

Generated embeddings with 'mean' pooling. Embedding dimension: 384


# Course Embeddings using PCA

In [19]:
pca = pca = PCA(n_components=2, random_state=42)
course_pca_result = pca.fit_transform(course_embeddings)
course_df['pca-one-course'] = course_pca_result[:, 0]
course_df['pca-two-course'] = course_pca_result[:, 1]


# Course Embeddings using t-SNE

In [20]:
tsne = TSNE(n_components=2, perplexity=30, max_iter=1000, random_state=42)
course_tsne_result = tsne.fit_transform(course_embeddings)
course_df['tsne-one-course'] = course_tsne_result[:, 0]
course_df['tsne-two-course'] = course_tsne_result[:, 1]


# UMAP

In [21]:
# Initialize UMAP with desired parameters
umap_projection = umap.UMAP(n_components=2, n_neighbors=15, min_dist=0.1)

# Fit and transform the course embeddings
course_umap_result = umap_projection.fit_transform(course_embeddings)

# Add UMAP results to your DataFrame
course_df['umap-one-course'] = course_umap_result[:, 0]
course_df['umap-two-course'] = course_umap_result[:, 1]


# Connect the Programs and Courses using Embeddings

In [22]:
# Ensure consistent data types for merge keys
program_course_df['courseId'] = program_course_df['courseId'].astype(str)
course_df['id'] = course_df['id'].astype(str)

program_course_df['programId'] = program_course_df['programId'].astype(str)
program_df['id'] = program_df['id'].astype(str)

In [23]:
# Merge DataFrames with suffixes to differentiate columns
combined_df = pd.merge(
    program_course_df,
    program_df,
    left_on='programId',
    right_on='id',
    how='left',
    suffixes=('_program_course', '_program')
)

# Print columns after first merge
print("Columns after merging program_course_df and program_df:")
print(combined_df.columns.tolist())


Columns after merging program_course_df and program_df:
['createdAt_program_course', 'updatedAt_program_course', 'typicalSessionIndex', 'courseId', 'programId', 'type', 'code', 'credits', 'horaireCoursPdfJson', 'planificationPdfJson', 'createdAt_program', 'updatedAt_program', 'title', 'url', 'cycle', 'id', 'vector', 'pca-one-program', 'pca-two-program', 'tsne-one-program', 'tsne-two-program', 'umap-one-program', 'umap-two-program']


In [24]:

# Merge with course_df
combined_df = pd.merge(
    combined_df,
    course_df,
    left_on='courseId',
    right_on='id',
    how='left',
    suffixes=('', '_course')
)

# Print columns after second merge
print("Columns after merging with course_df:")
print(combined_df.columns.tolist())


Columns after merging with course_df:
['createdAt_program_course', 'updatedAt_program_course', 'typicalSessionIndex', 'courseId', 'programId', 'type', 'code', 'credits', 'horaireCoursPdfJson', 'planificationPdfJson', 'createdAt_program', 'updatedAt_program', 'title', 'url', 'cycle', 'id', 'vector', 'pca-one-program', 'pca-two-program', 'tsne-one-program', 'tsne-two-program', 'umap-one-program', 'umap-two-program', 'code_course', 'title_course', 'description', 'credits_course', 'createdAt', 'updatedAt', 'id_course', 'cycle_course', 'vector_course', 'pca-one-course', 'pca-two-course', 'tsne-one-course', 'tsne-two-course', 'umap-one-course', 'umap-two-course']


In [25]:
# Optional: Rename columns for clarity
combined_df.rename(columns={
    'title': 'title_program',
    'code': 'code_program',
    'cycle': 'cycle_program',
    'credits': 'credits_program',
    'horaireCoursPdfJson': 'horaireCoursPdfJson_program',
    'title_course': 'title_course',
    'code_course': 'code_course',
    'cycle_course': 'cycle_course',
    'credits_course': 'credits_course',
    'description': 'description_course'
}, inplace=True)

# Updated list of columns to include in the combined text
columns_to_include = [
    'programId', 'courseId', 'type',
    'title_program', 'code_program', 'cycle_program', 'credits_program', 'horaireCoursPdfJson_program',
    'title_course', 'code_course', 'cycle_course', 'credits_course', 'description_course'
]


In [26]:
# Ensure all columns are strings and handle NaN values
for col in columns_to_include:
    combined_df[col] = combined_df[col].astype(str).fillna('')

# Concatenate the columns into a single string for each record
combined_texts = combined_df[columns_to_include].apply(lambda x: ' '.join(x), axis=1)

# Generate embeddings for the combined texts
combined_embeddings = generate_embeddings(combined_texts.tolist(), tokenizer, model)

# Add embeddings to combined_df
combined_df['vector'] = list(combined_embeddings)


Generated embeddings with 'mean' pooling. Embedding dimension: 384


In [27]:
pca = pca = PCA(n_components=2, random_state=42)
combined_pca_result = pca.fit_transform(combined_embeddings)
combined_df['pca-one-combined'] = combined_pca_result[:, 0]
combined_df['pca-two-combined'] = combined_pca_result[:, 1]

In [28]:
tsne = TSNE(n_components=2, perplexity=50, max_iter=1000, random_state=42)
combined_tsne_result = tsne.fit_transform(combined_embeddings)
combined_df['tsne-one-combined'] = combined_tsne_result[:, 0]
combined_df['tsne-two-combined'] = combined_tsne_result[:, 1]


In [29]:
# Check the number of unique programs
num_unique_programs = combined_df['title_program'].nunique()
print(f"Number of unique programs: {num_unique_programs}")

# Initialize UMAP with desired parameters
umap_projection = umap.UMAP(n_components=2, n_neighbors=15, min_dist=0.1)

# Fit and transform the combined embeddings
combined_umap_result = umap_projection.fit_transform(combined_embeddings)

# Add UMAP results to the DataFrame
combined_df['umap-one-combined'] = combined_umap_result[:, 0]
combined_df['umap-two-combined'] = combined_umap_result[:, 1]


Number of unique programs: 44


# Implement KNN to Find Similar Courses

In [30]:
knn = NearestNeighbors(metric='cosine', algorithm='auto', n_neighbors=10)
knn.fit(vectors)

def find_similar_courses(index, n_neighbors=5):
    course_embedding = vectors[index].reshape(1, -1)
    distances, indices = knn.kneighbors(course_embedding, n_neighbors=n_neighbors+1)
    similar_courses = []
    for i in range(1, len(indices[0])):
        idx = indices[0][i]
        similar_courses.append({
            'course_id': course_df.iloc[idx]['id'],
            'title': course_df.iloc[idx]['title'],
            'cycle': course_df.iloc[idx]['cycle'],
            'distance': distances[0][i]
        })
        
    return similar_courses


In [31]:
# Example usage
index_to_query = 2
similar_courses = find_similar_courses(index_to_query, n_neighbors=5)

queried_course = course_df.iloc[index_to_query]
print(f"Courses similar to '{queried_course['title']}' in cycle '{queried_course['cycle']} (ID: {queried_course['description']}):")
for course in similar_courses:
    print(f"- {course['title']} (Cycle: {course['cycle']}, Description: {course['course_id']}, Distance: {course['distance']:.4f})")

Courses similar to 'Analytiques des données massives' in cycle '2 (ID: Ce cours présente les concepts pour effectuer une analyse statistique de très grands ensembles de données qui ne tiennent pas sur un seul ordinateur. Ce cours vous permettra développer vos connaissances en analyse de données massives et améliorerez vos compétences en programmation et en mathématiques. Vous apprendrez à utiliser des outils analytiques essentiels pour l’analyse statistique des données massives. Plusieurs problèmes applicatifs seront étudiés et différentes...):
- Introduction à l’analyse des mégadonnées (Cycle: 1, Description: 352303, Distance: 0.1462)
- Maîtrise statistique des procédés (Cycle: 1, Description: 351867, Distance: 0.2465)
- Analyse de données et systèmes prédictifs (Cycle: 2, Description: 350753, Distance: 0.2503)
- Informatique et statistiques appliquées (Cycle: 1, Description: 352663, Distance: 0.2550)
- Entrepôts de données et intelligence d’affaires (Cycle: 2, Description: 353322, Di

# Recommandation

In [32]:
# Extract embeddings from combined_df
combined_vectors = np.stack(combined_df['vector'].values)

# Initialize KNN with cosine distance
knn_combined = NearestNeighbors(metric='cosine', algorithm='auto', n_neighbors=10)

# Fit KNN on combined embeddings
knn_combined.fit(combined_vectors)

def find_similar_combinations(index, n_neighbors=5):
    """
    Find similar program-course combinations based on embeddings.

    Parameters:
    - index (int): Index of the record in combined_df to query.
    - n_neighbors (int): Number of similar combinations to retrieve.

    Returns:
    - List of dictionaries containing details of similar combinations.
    """
    # Validate the index
    if index < 0 or index >= len(combined_df):
        raise IndexError("Index out of bounds for combined_df.")

    # Get the embedding for the specified index
    combination_embedding = combined_vectors[index].reshape(1, -1)

    # Find nearest neighbors (including the query itself)
    distances, indices = knn_combined.kneighbors(combination_embedding, n_neighbors=n_neighbors + 1)

    similar_combinations = []
    for i in range(1, len(indices[0])):  # Start from 1 to exclude the query itself
        idx = indices[0][i]
        similar_combinations.append({
            'program_id': combined_df.iloc[idx]['programId'],
            'program_title': combined_df.iloc[idx]['title_program'],
            'course_id': combined_df.iloc[idx]['courseId'],
            'course_title': combined_df.iloc[idx]['title_course'],
            'cycle': combined_df.iloc[idx]['cycle_course'],
            'distance': distances[0][i]
        })
    return similar_combinations


In [33]:
# Example usage
index_to_query = 2  # Change this index based on your data

try:
    # Find similar program-course combinations
    similar_combinations = find_similar_combinations(index_to_query, n_neighbors=5)

    # Get details of the queried combination
    queried_combination = combined_df.iloc[index_to_query]

    print(f"Program-Course combinations similar to '{queried_combination['title_program']}' "
          f"program and '{queried_combination['title_course']}' course "
          f"in cycle '{queried_combination['cycle_course']}' (Program ID: {queried_combination['programId']}, "
          f"Course ID: {queried_combination['courseId']}):\n")

    for combo in similar_combinations:
        print(f"- Program: {combo['program_title']} (ID: {combo['program_id']}), "
              f"Course: {combo['course_title']} (ID: {combo['course_id']}), "
              f"Cycle: {combo['cycle']}, Distance: {combo['distance']:.4f}")
except IndexError as e:
    print(f"Error: {e}")


Program-Course combinations similar to 'Certificat en gestion immobilière' program and 'Gestion des actifs immobiliers' course in cycle '1' (Program ID: 182976, Course ID: 352245):

- Program: Programme court en gestion immobilière (ID: 183016), Course: Gestion des actifs immobiliers (ID: 352245), Cycle: 1, Distance: 0.0207
- Program: Programme court en gestion immobilière (ID: 183016), Course: Fondements et immobilier (ID: 352233), Cycle: 1, Distance: 0.1031
- Program: Certificat en gestion immobilière (ID: 182976), Course: Gestion du personnel et relations industrielles (ID: 351827), Cycle: 1, Distance: 0.1234
- Program: Programme court en gestion immobilière (ID: 183016), Course: Gestion du personnel et relations industrielles (ID: 351827), Cycle: 1, Distance: 0.1351
- Program: Programme court en gestion immobilière (ID: 183016), Course: Planification et contrôle de projets (ID: 351065), Cycle: 1, Distance: 0.1416


# One program recommandation of courses

In [34]:
def tokenize_sentences(text):
    """
    Tokenizes the input text into sentences using NLTK's sent_tokenize for French.
    """
    sentences = sent_tokenize(text, language='french')
    return ' '.join(sentences)

def preprocess_text_input(text):
    """
    Preprocesses the input text by removing stopwords and tokenizing.
    """
    sentences = tokenize_sentences(text)
    preprocessed = preprocess_text(sentences)
    return preprocessed

def embed_text(text):
    """
    Generates embedding for a single piece of text.
    """
    model.eval()
    with torch.no_grad():
        tokens = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512)
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        model.to(device)
        tokens = {k: v.to(device) for k, v in tokens.items()}
        outputs = model(**tokens)
        embedding = outputs.last_hidden_state.mean(dim=1).cpu().numpy()
    return embedding

def find_similar_combinations_within_program(program_id, text_input=None, n_neighbors=5):
    """
    Find similar program-course combinations within a specific program based on an optional text input.

    Parameters:
    - program_id (str): Identifier of the program to restrict the search.
    - text_input (str): Optional text input to base similarity on.
    - n_neighbors (int): Number of similar combinations to retrieve.

    Returns:
    - List of dictionaries containing details of similar combinations.
    """
    # Step 1: Filter the DataFrame to include only the specified program
    program_subset = combined_df[combined_df['programId'] == program_id]

    if program_subset.empty:
        raise ValueError(f"No data found for program ID: {program_id}")

    # Step 2: Extract embeddings for the subset
    subset_vectors = np.stack(program_subset['vector'].values)

    # Step 3: Initialize and fit KNN on the subset
    knn_program = NearestNeighbors(metric='cosine', algorithm='brute')
    knn_program.fit(subset_vectors)

    # Step 4: Handle text input (if provided)
    if text_input:
        # Preprocess and embed the input text
        preprocessed_text = preprocess_text_input(text_input)
        input_embedding = embed_text(preprocessed_text)
    else:
        # If no text input, default to the first combination in the subset
        input_embedding = subset_vectors[0].reshape(1, -1)

    # Step 5: Perform KNN search
    distances, indices = knn_program.kneighbors(input_embedding, n_neighbors=n_neighbors + 1)

    # Step 6: Compile similar combinations (excluding the query itself)
    similar_combinations = []
    for i in range(1, len(indices[0])):  # Start from 1 to exclude the query itself
        idx = indices[0][i]
        combo = program_subset.iloc[idx]
        similar_combinations.append({
            'program_id': combo['programId'],
            'program_title': combo['title_program'],
            'course_id': combo['courseId'],
            'course_title': combo['title_course'],
            'cycle': combo['cycle_course'],
            'distance': distances[0][i]
        })

    return similar_combinations

In [35]:
def visualize_similar_combinations_within_program(program_id, text_input=None, n_neighbors=5):
    try:
        # Find similar combinations
        similar_combinations = find_similar_combinations_within_program(
            program_id=program_id,
            text_input=text_input,
            n_neighbors=n_neighbors
        )

        # Get the course IDs of similar combinations
        similar_course_ids = [combo['course_id'] for combo in similar_combinations]

        # Filter the DataFrame for the specified program
        program_subset = combined_df[combined_df['programId'] == program_id]

        # Create the base t-SNE scatter plot
        fig = px.scatter(
            program_subset,
            x='tsne-one-combined',
            y='tsne-two-combined',
            color='title_program',
            hover_data=['title_course'],
            title=f"'{program_id}' Embeddings with Similar Courses Highlighted",
            labels={
                'tsne-one-combined': 't-SNE Dimension 1',
                'tsne-two-combined': 't-SNE Dimension 2'
            }
        )

        # Highlight similar courses
        similar_df = program_subset[program_subset['courseId'].isin(similar_course_ids)]
        fig.add_trace(
            px.scatter(
                similar_df,
                x='tsne-one-combined',
                y='tsne-two-combined',
                hover_data=['title_course']
            ).data[0]
        )

        fig.show()
    except (IndexError, ValueError) as e:
        print(f"Error: {e}")


In [36]:
# Example usage
program_id_to_query = '182912'
text_input = "DevOps"

try:
    similar_combinations = find_similar_combinations_within_program(
        program_id=program_id_to_query,
        text_input=text_input,
        n_neighbors=5
    )

    queried_program = combined_df[combined_df['programId'] == program_id_to_query].iloc[0]
    for combo in similar_combinations:
        print(f"- Course: {combo['course_title']} (ID: {combo['course_id']}), "
              f"Cycle: {combo['cycle']}, Distance: {combo['distance']:.4f}")
    
    # Optional: Visualize the results
    visualize_similar_combinations_within_program(program_id_to_query, text_input, n_neighbors=5)
except (IndexError, ValueError) as e:
    print(f"Error: {e}")

- Course: Intégrité intellectuelle (ID: 349682), Cycle: 1, Distance: 0.7037
- Course: Conception orientée objet (ID: 352405), Cycle: 1, Distance: 0.7278
- Course: Logique et mathématiques discrètes (ID: 352637), Cycle: 1, Distance: 0.7314
- Course: Progiciels de gestion intégrée (ID: 352039), Cycle: 1, Distance: 0.7314
- Course: Environnement, technologie et société (ID: 354038), Cycle: 1, Distance: 0.7466


In [37]:
import gradio as gr

# Map program titles to program IDs
program_title_to_id = dict(zip(combined_df['title_program'], combined_df['programId']))

# Map course titles to course IDs
course_title_to_id = dict(zip(combined_df['title_course'], combined_df['courseId']))

def find_and_visualize_similar_combinations(selected_program_title, text_input, n_neighbors=5):
    """
    Function to find similar combinations and visualize the results.
    """
    # Get program ID based on the selected title
    program_id = program_title_to_id.get(selected_program_title)

    if not program_id:
        return "Invalid program selection. Please choose a valid program."

    try:
        # Find similar combinations within the program
        similar_combinations = find_similar_combinations_within_program(
            program_id=program_id,
            text_input=text_input,
            n_neighbors=n_neighbors
        )


        # Display results
        result_text = f"'{selected_program_title}':\n\n"
        for combo in similar_combinations:
            result_text += (
                f"- Course: {combo['course_title']} (Cycle: {combo['cycle']}), "
                f"Distance: {combo['distance']:.4f}\n"
            )
        return result_text

    except ValueError as e:
        return str(e)

# Define Gradio interface
program_dropdown = gr.Dropdown(
    label="Select a Program",
    choices=list(program_title_to_id.keys())
)

text_input = gr.Textbox(
    label="Input Text for Similarity Search",
    placeholder="Enter a description or text to find similar courses..."
)

n_neighbors_slider = gr.Slider(
    label="Number of Similar Courses to Find",
    minimum=1,
    maximum=10,
    step=1,
    value=5,
)

output_text = gr.Textbox(
    label="Similarity Search Results"
)

# Build Gradio interface
interface = gr.Interface(
    fn=find_and_visualize_similar_combinations,
    inputs=[program_dropdown, text_input, n_neighbors_slider],
    outputs=output_text,
    live=True,
    title="Program-Course Similarity Search",
    description="Select a program, optionally enter text, and find similar course combinations."
)

# Launch the Gradio app
interface.launch()


* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
